In [7]:
from datasets import Dataset, Image, Features, Value
import json
from PIL import Image as PILImage
import os
import glob


def generate_examples():
    for data in annotations:
        image_path = os.path.join("Spatial-MM/data", data["image_name"])
        try:
            image = PILImage.open(image_path).convert("RGB")  # Load image as RGB
        except Exception as e:
            print("Error loading image: ", image_path)
            continue
        yield {
            "image": image,
            "question": data["question"],
            "answer": data["answer"],
        }


    # Load JSON annotations from multiple files
annotations = []
json_files = glob.glob("Spatial-MM/data/Spatial_MM_CoT/*.json")
for json_file in json_files:
	with open(json_file, "r") as f:
		annotations.extend(json.load(f))

print(f"Loaded {len(annotations)} annotations")

# Define dataset features (adjust based on your JSON structure)
# features = Features(
# 	{
# 		"image": Image(),
# 		"question": Value("string"),
# 		"answer": Value("string"),
# 	}
# )

# # Create the dataset
# dataset = Dataset.from_generator(
# 	generate_examples,
# 	features=features,
# )

# dataset
    # dataset.push_to_hub("spatial_mm", private=True)  # Push to the Hub



Loaded 2211 annotations


In [ ]:
import shutil
import os

source_dirs = ['Spatial-MM/data/Spatial_MM_Obj', 'Spatial-MM/data/Spatial_MM_CoT']
target_dir = 'Spatial-MM/data/images'

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

for source in source_dirs:
	for filename in os.listdir(source):
		source_file = os.path.join(source, filename)
		if os.path.isfile(source_file):
			shutil.move(source_file, target_dir)

In [ ]:
image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]

source_dirs = ["Spatial-MM/data/Spatial_MM_Obj", "Spatial-MM/data/Spatial_MM_CoT"]
target_dir = "Spatial-MM/data/image_dir"

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

for source in source_dirs:
	for filename in os.listdir(source):
		if os.path.splitext(filename)[1].lower() in image_extensions:
			shutil.move(os.path.join(source, filename), target_dir)


In [10]:
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
image_files = [
	f for f in os.listdir('Spatial-MM/data/Spatial_MM_Obj')
	if os.path.splitext(f)[1].lower() in image_extensions
]
print(f"Number of images: {len(image_files)}")

Number of images: 5702


In [1]:
from datasets import Dataset, Image, Features, Value
import json
from PIL import Image as PILImage
import os
import glob


def generate_examples():
    for data in annotations:
        image_path = os.path.join("data/egobrientbench", data["image"])
        try:
            image = PILImage.open(image_path).convert("RGB")  # Load image as RGB
        except Exception as e:
            print("Error loading image: ", image_path)
            continue
        yield {
            "image": image,
            "question": data["question"],
            "answer": data["label"],
        }

    # Load JSON annotations from multiple files


annotations = []
json_files = glob.glob("data/egobrientbench/benchmark.json")
for json_file in json_files:
    with open(json_file, "r") as f:
        annotations.extend(json.load(f))

print(f"Loaded {len(annotations)} annotations")

# Define dataset features (adjust based on your JSON structure)
features = Features(
    {
        "image": Image(),
        "question": Value("string"),
        "answer": Value("string"),
    }
)

# Create the dataset
dataset = Dataset.from_generator(
    generate_examples,
    features=features,
)

dataset
# dataset.push_to_hub("spatial_mm", private=True)  # Push to the Hub


Loaded 33460 annotations


Dataset({
    features: ['image', 'question', 'answer'],
    num_rows: 33460
})

In [3]:
dataset.push_to_hub("stogian/egoorientbench", private=True )  # Push to the Hub

Uploading the dataset shards:   0%|          | 0/9 [00:00<?, ?it/s]

Map:   0%|          | 0/3718 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

Map:   0%|          | 0/3718 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/3717 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

Map:   0%|          | 0/3717 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/38 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/stogian/egoorientbench/commit/555bae171477e4755b0193eed2cc9f6c42b4d6e5', commit_message='Upload dataset', commit_description='', oid='555bae171477e4755b0193eed2cc9f6c42b4d6e5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/stogian/egoorientbench', endpoint='https://huggingface.co', repo_type='dataset', repo_id='stogian/egoorientbench'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import notebook_login

!pip install transformers


model_name = "bert-base-uncased"
model = AutoModel.from_pretrained(model_name, cache_dir="../bin/models/"+model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="../bin/models/"+model_name)
notebook_login()